In [1]:
import numpy as np
from simulation_framework.item_response_simulation import item_response_simulation
from simulation_framework.simulate_competency import respondent_population
from simulation_framework.simulate_responses import response_simulation
from scipy.stats import multivariate_normal
import models
import em_algorithm
import pandas as pd
import time
from girth import multidimensional_twopl_mml
from girth import twopl_mml
import cma
import scipy

C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models
C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models


In [2]:
def params_to_vector(model):
    item_dimension = model.item_dimension
    latent_dimension = model.latent_dimension
    A_flat = model.item_parameters["discrimination_matrix"].flatten()
    A_flat = A_flat[A_flat != 0]
    A_cut = len(A_flat)
    delta_cut = A_cut+item_dimension
    delta = model.item_parameters["intercept_vector"]
    sigma_flat = model.person_parameters["covariance"][np.triu_indices(
            model.latent_dimension, k=1)]
    return(np.concatenate((A_flat, delta, sigma_flat), axis=0))

def vector_to_params(vector, model):
    item_dimension = model.item_dimension
    latent_dimension = model.latent_dimension 
    q_matrix = model.item_parameters["q_matrix"] 
    q_flat = q_matrix.flatten()
    A_cut = len(q_flat[q_flat != 0])
    delta_cut = A_cut+item_dimension
    A = vector[0:A_cut]
    A = model.fill_zero_discriminations(A).reshape((item_dimension, latent_dimension))
    delta = vector[A_cut:delta_cut]
    item_parameters = {"discrimination_matrix": A, "intercept_vector": delta}
    corr = vector[delta_cut: len(vector)]
    sigma = model.corr_to_sigma(corr, False)
    return({"item_parameters": item_parameters, "person_parameters": {"covariance": sigma}})

def direct_marginal_optimization(model, response_data):
    #Get initial parameters
    x0 = params_to_vector(model)
    response_data = response_data.to_numpy()
    #Optimize with GA
    def marginal_l_func(input_vector):
        parameters = vector_to_params(input_vector, model)
        try:
            model.set_parameters(parameters)
        except Exception:
            return(np.inf)
        result = -1*model.marginal_response_loglikelihood(response_data)
        return(result)
    es = cma.CMAEvolutionStrategy(x0=x0, sigma0=0.5)
    es.optimize(marginal_l_func, maxfun=100000)
    #Get result
    result = es.result.xfavorite
    return(vector_to_params(result, model))

In [3]:
from models.mirt_2pl import mirt_2pl

def rmse(y_pred: np.array, y_true: np.array) -> float:
    MSE = np.square(np.subtract(y_pred.flatten(), y_true.flatten())).mean()
    RMSE = np.sqrt(MSE)
    return(float(RMSE))

def experiment_performance(estimated_parameter_dict, real_parameter_dict):
    res_dict = {}
    if "item_parameters" in estimated_parameter_dict.keys():
        A_pred = estimated_parameter_dict["item_parameters"]["discrimination_matrix"]
        delta_pred = estimated_parameter_dict["item_parameters"]["intercept_vector"]

        A_true = real_parameter_dict["item_parameters"]["discrimination_matrix"]
        delta_true = real_parameter_dict["item_parameters"]["intercept_vector"]

        print("Absolute diff in A:")
        print(np.abs(A_true-A_pred))

        print("Absolute diff in delta:")
        print(np.abs(delta_true-delta_pred))

        res_dict["rmse_A"] = rmse(A_pred, A_true)
        res_dict["rmse_delta"] = rmse(delta_true, delta_pred)

    if "person_parameters" in estimated_parameter_dict.keys():
        sigma_pred = estimated_parameter_dict["person_parameters"]["covariance"]
        sigma_true = real_parameter_dict["person_parameters"]["covariance"]

        print("Absolute diff in sigma:")
        print(np.abs(sigma_true-sigma_pred)) # TODO: Evtl. die Hauptdiagonale ausschließen!

        res_dict["rmse_sigma"] = rmse(sigma_true, sigma_pred) 
    if len(res_dict.keys())==0:
        raise Exception("No performance to calculate")
    return(res_dict)

def standardize_parameters(parameter_dict):
        covariance = parameter_dict["person_parameters"]["covariance"]
        A = parameter_dict["item_parameters"]["discrimination_matrix"]
        if len(covariance.shape)<=1:
            correlation_matrix  = np.array([[1]])
            inv_sqrt_cov = np.array([[np.sqrt(1/covariance)]])
            A = np.multiply(A, inv_sqrt_cov)
        else:
            sd_vector = np.sqrt(covariance.diagonal())
            inv_sd_matrix = np.linalg.inv(np.diag(sd_vector))
            correlation_matrix = np.dot(
            np.dot(inv_sd_matrix, covariance), inv_sd_matrix)
            A = np.dot(A, np.linalg.inv(inv_sd_matrix))
        parameter_dict["item_parameters"]["discrimination_matrix"] = A
        parameter_dict["person_parameters"]["covariance"] = correlation_matrix
        return(parameter_dict)

def create_parameter_dict(estimated_early_parameters, real_early_parameters, estimated_late_parameters, real_late_parameters):
    parameter_dict = {"real_early_parameters": real_early_parameters,
                   "estimated_early_parameters": estimated_early_parameters}
    return(parameter_dict)


def create_performance_dict(parameter_dict, run_dict, sample=None, baselines=None, model=None):
    result_dict = parameter_dict
    result_dict["sample"] = sample
    #Model Performance
    result_dict["early_performance"] = {}
    result_dict["early_performance"]["rmse"] = experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=result_dict["estimated_early_parameters"])
    #Baseline's Performance
    baselines["early_initial"]["performance"] = {"rmse": experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=baselines["early_initial"]["parameters"])}
    if "girth" in baselines.keys():
        girth_data = result_dict["sample"]["early_responses"].to_numpy().transpose()
        if model.latent_dimension == 1:
            girth_estimates = twopl_mml(girth_data)
        else:
            girth_estimates = multidimensional_twopl_mml(girth_data, n_factors=model.latent_dimension)
        girth_item_parameters = {"discrimination_matrix": girth_estimates["Discrimination"], "intercept_vector": girth_estimates["Difficulty"]}
        girth_parameters = {"item_parameters": girth_item_parameters}
        girth_estimated_covariance = np.cov(girth_estimates["Ability"], rowvar=True)
        girth_parameters["person_parameters"] = {"covariance": girth_estimated_covariance}
        girth_parameters = standardize_parameters(girth_parameters)
        girth_performance_rmse = experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=girth_parameters)
        baselines["girth"]["parameters"] = girth_parameters
        baselines["girth"]["performance"] = {"rmse": girth_performance_rmse}

    if "early_direct" in baselines.keys():
        dir_model = mirt_2pl(item_dimension=model.item_dimension, latent_dimension=model.latent_dimension, Q=model.item_parameters["q_matrix"])
        direct_early_item_parameters = direct_marginal_optimization(dir_model, response_data=sample["early_responses"])
        direct_early_performance_rmse = experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=direct_early_item_parameters)
        baselines["early_direct"]["parameters"] = direct_early_item_parameters
        baselines["early_direct"]["performance"] = {"rmse": direct_early_performance_rmse}

    result_dict["baselines"] = baselines
    likelihood = calculate_marginal_likelihoods(model=model, response_data=sample["early_responses"], real_parameters=result_dict["real_early_parameters"],
                                                initial_parameters=baselines["early_initial"]["parameters"], estimated_parameters=result_dict["estimated_early_parameters"])
    result_dict["early_performance"]["marginal_likelihood"] = likelihood
    result_dict["early_performance"]["run"] = run_dict["early"]
    return(result_dict)

def calculate_marginal_likelihoods(model, response_data, real_parameters, initial_parameters, estimated_parameters):
    model.set_parameters(initial_parameters)
    initial_marginal_likelihood = model.marginal_response_loglikelihood(response_data=response_data.to_numpy())
    model.set_parameters(real_parameters)
    optimal_marginal_likelihood = model.marginal_response_loglikelihood(response_data=response_data.to_numpy())
    model.set_parameters(estimated_parameters)
    marginal_likelihood_estimated = model.marginal_response_loglikelihood(response_data=response_data.to_numpy())
    likelihood_dict = {"optimal": optimal_marginal_likelihood, "estimated": marginal_likelihood_estimated, "initial": initial_marginal_likelihood}
    return(likelihood_dict)

In [4]:
def direct_optimization_benchmark():
    latent_dimension = 2
    item_dimension = 6
    sample_size=100

    #Define Population
    real_latent_cov = np.array([[1,0.2],
                    [0.2,1]])

    latent_distribution = multivariate_normal(mean=np.array([0,0]), cov=real_latent_cov)
    population = respondent_population(latent_dimension=latent_dimension, latent_distribution=latent_distribution)

    #Define Test
    A = np.array([[0.5,0],
                  [2,0],
                  [0,0.4],
                  [0.7,0.6],
                  [2, 0.9],
                  [0, 1.5]])
    Q = np.array([[1,0],
                  [1,0],
                  [0,1],
                  [1,1],
                  [1,1],
                  [0,1]])

    delta = np.array([0, 0.5, 1, 0, 1.5, 0.9])
    early_item_parameters = {"discrimination_matrix": A, "intercept_vector": delta, "q_matrix": Q, "item_dimension": item_dimension, "latent_dimension": latent_dimension}
    real_early_parameters = {"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}}

    #Sample responses
    response_simulation_obj = response_simulation(population=population, item_dimension = item_dimension, early_item_params=early_item_parameters)
    sample = response_simulation_obj.sample(sample_size)

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=2, item_dimension=6, Q=Q)
    model.initialize_from_responses(response_data=sample["early_responses"])

    estimated_params = direct_marginal_optimization(model, response_data=sample["early_responses"])
    return(estimated_params)


## Experiment 0: MIRT-2PL Performance Benchmark

In [5]:
def mirt_performance_benchmark() -> dict:
    latent_dimension = 2
    item_dimension = 6
    sample_size=100

    #Define Population
    real_latent_cov = np.array([[1,0.2],
                    [0.2,1]])

    latent_distribution = multivariate_normal(mean=np.array([0,0]), cov=real_latent_cov)
    population = respondent_population(latent_dimension=latent_dimension, latent_distribution=latent_distribution)

    #Define Test
    A = np.array([[0.5,0],
                  [2,0],
                  [0,0.4],
                  [0.7,0.6],
                  [2, 0.9],
                  [0, 1.5]])
    Q = np.array([[1,0],
                  [1,0],
                  [0,1],
                  [1,1],
                  [1,1],
                  [0,1]])

    delta = np.array([0, 0.5, 1, 0, 1.5, 0.9])
    early_item_parameters = {"discrimination_matrix": A, "intercept_vector": delta, "q_matrix": Q, "item_dimension": item_dimension, "latent_dimension": latent_dimension}
    real_early_parameters = {"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}}

    #Sample responses
    response_simulation_obj = response_simulation(population=population, item_dimension = item_dimension, early_item_params=early_item_parameters)
    sample = response_simulation_obj.sample(sample_size)

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=2, item_dimension=6, Q=Q)
    model.initialize_from_responses(response_data=sample["early_responses"])
    initial_early_parameters = model.get_parameters()
    e_step = em_algorithm.e_step_ga_mml(model=model)
    m_step = em_algorithm.m_step_ga_mml(model)
    em = em_algorithm.em_algo(e_step=e_step, m_step=m_step, model=model)

    #Fit Model
    start_time = time.time()
    em.fit(sample["early_responses"], max_iter=50)
    run_time =  (time.time() - start_time)

    #Create Baselines
    baselines = {"early_initial": {"parameters": initial_early_parameters}, "girth": {}, "early_direct": {}}
    
    #Create results
    early_estimated_parameters = em.model.get_parameters()

    parameter_dict = create_parameter_dict(estimated_early_parameters=early_estimated_parameters,
                          real_early_parameters = real_early_parameters,
                          estimated_late_parameters=None, real_late_parameters=None)
    run_dict = {"early": {"runtime": run_time,
                "number_steps": em.n_steps}}
    performance_dict = create_performance_dict(parameter_dict=parameter_dict, run_dict=run_dict, sample=sample, baselines=baselines, model=model)
    return(performance_dict)

In [6]:

def print_result(result_dict, description=""):
    #Description
    print("------------------------------------")
    print("##### Results for {0}".format(description))
    #Experiment Properties:
    latent_dimension = result_dict["sample"]["latent_dimension"]
    item_dimension = result_dict["sample"]["item_dimension"]
    sample_size = result_dict["sample"]["sample_size"]
    print("Latent dimension: {0},  Item dimension: {1}, sample size {2} \\".format(latent_dimension, item_dimension, sample_size))
    #Performance/time
    ep_dict = result_dict["early_performance"]
    runtime = np.round(ep_dict["run"]["runtime"], 2)
    steps = ep_dict["run"]["number_steps"]
    print("Runtime: {0} seconds, {1} steps, {2} seconds per step \\".format(runtime, steps, np.round(runtime/steps, 2)))
    #Performance/results
    l_optimal = np.round(ep_dict["marginal_likelihood"]["optimal"], 2)
    l_estimated = np.round(ep_dict["marginal_likelihood"]["estimated"], 2)
    l_real = np.round(ep_dict["marginal_likelihood"]["initial"], 2)
    print("Optimal marginal Likelihood: {0}, Estimated: {1}, Initial {2}".format(l_optimal,l_estimated,l_real))
    #print("Performance: rmse-mean = {0} \\".format(np.round(np.mean(np.array(list(result_dict["early_performance"].values()))), 4)))
    rmse_model = ep_dict["rmse"]
    
    rmse_frame = pd.DataFrame(columns=["rmse_A", "rmse_delta", "rmse_sigma"])
    rmse_frame = rmse_frame.append(rmse_model, ignore_index=True)
    index = ["estimated"]
    for baseline in list(result_dict["baselines"].keys()):
        rmse_baseline = result_dict["baselines"][baseline]["performance"]["rmse"]
        rmse_frame = rmse_frame.append(rmse_baseline, ignore_index=True)
        index.append(baseline)
    rmse_frame.index = index
    print(rmse_frame.to_markdown())
                                             

In [7]:
performance_dict = mirt_performance_benchmark()

EM Iteration 2
Current Monte Carlo Sample size: 300


c:\Users\Jesper\anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Step: 2: current parameter_diff: 1.7696927079916756, current marginal loglikelihood: -370.58996916417544
EM Iteration 3
Current Monte Carlo Sample size: 300
Step: 3: current parameter_diff: 0.42359683758172406, current marginal loglikelihood: -370.10636148292815
EM Iteration 4
Current Monte Carlo Sample size: 360
Step: 4: current parameter_diff: 0.373014145884365, current marginal loglikelihood: -369.74380602753797
EM Iteration 5
Current Monte Carlo Sample size: 360
Step: 5: current parameter_diff: 0.5249486480225427, current marginal loglikelihood: -369.5677709304813
EM Iteration 6
Current Monte Carlo Sample size: 432
Step: 6: current parameter_diff: 0.20546962329286378, current marginal loglikelihood: -369.1280354609925
EM Iteration 7
Current Monte Carlo Sample size: 432
Step: 7: current parameter_diff: 0.12076279038520388, current marginal loglikelihood: -369.0871680390001
EM Iteration 8
Current Monte Carlo Sample size: 518
Step: 8: current parameter_diff: 0.06353642408981242, curre

c:\Users\Jesper\anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [3] are not finite but [inf]. (class=CMAEvolutionStrategy method=ask iteration=3)
  warnings.warn(msg + ' (' +
c:\Users\Jesper\anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [11] are not finite but [inf]. (class=CMAEvolutionStrategy method=ask iteration=5)
  warnings.warn(msg + ' (' +
c:\Users\Jesper\anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [1] are not finite but [inf]. (class=CMAEvolutionStrategy method=ask iteration=6)
  warnings.warn(msg + ' (' +


[[1.         1.71821313]
 [1.71821313 1.        ]]
[[1.         1.17121982]
 [1.17121982 1.        ]]
[[1.         1.06951427]
 [1.06951427 1.        ]]


c:\Users\Jesper\anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [4] are not finite but [inf]. (class=CMAEvolutionStrategy method=ask iteration=7)
  warnings.warn(msg + ' (' +
c:\Users\Jesper\anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [3] are not finite but [inf]. (class=CMAEvolutionStrategy method=ask iteration=8)
  warnings.warn(msg + ' (' +
c:\Users\Jesper\anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [3] are not finite but [inf]. (class=CMAEvolutionStrategy method=ask iteration=9)
  warnings.warn(msg + ' (' +


   53    636 3.680830559292414e+02 2.5e+00 8.05e-02  5e-02  9e-02 0:03.3
  100   1200 3.678035683980852e+02 3.7e+00 2.87e-02  2e-02  3e-02 0:06.0
  187   2244 3.676182653719185e+02 6.5e+00 7.40e-03  3e-03  1e-02 0:11.0
  200   2400 3.677084939338521e+02 6.5e+00 5.68e-03  3e-03  7e-03 0:11.7
  300   3600 3.675116698032161e+02 8.9e+00 3.61e-03  1e-03  4e-03 0:17.6
  400   4800 3.676025693976969e+02 1.6e+01 8.98e-03  3e-03  1e-02 0:23.5
  500   6000 3.674863026095088e+02 2.2e+01 2.49e-03  7e-04  3e-03 0:29.2
  600   7200 3.676645762446752e+02 2.7e+01 2.06e-03  6e-04  2e-03 0:35.0


In [8]:
print_result(performance_dict, "Quasi Monte-Carlo with increasing sample-size based on ttest")

------------------------------------
##### Results for Quasi Monte-Carlo with increasing sample-size based on ttest
Latent dimension: 2,  Item dimension: 6, sample size 100 \
Runtime: 4.15 seconds, 16 steps, 0.26 seconds per step \
Optimal marginal Likelihood: -367.97, Estimated: -363.66, Initial -370.45
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.361749 |     0.369746 |    0.0462448 |
| early_initial | 0.509902 |     0.383683 |    0.212132  |
| girth         | 0.90968  |     0.367442 |    0.0977334 |
| early_direct  | 0.362239 |     0.343475 |    0.128536  |


------------------------------------
##### Results for Naive, all-Genetic, no vectorization
Runtime: 29100 seconds, 100+ steps, 291 seconds per step\
Performance: rmse-mean = 0.5896464054719196

------------------------------------
##### Results for Vectorized q_item
Runtime: 613.3871161937714 seconds, 100+ steps, 6.0731397642947655 seconds per step \
Performance: rmse-mean = 0.6466789259215839

------------------------------------
##### Results for More precise results through higher N and pop_size
Runtime: 2239.073846578598 seconds, 100+ steps, 22.169047985926714 seconds per step \
Performance: rmse-mean = 0.6384660709857508

------------------------------------
##### Results for Debug GA sorting, Add Likelihood-based stopping criterion
Runtime: 135.11203932762146 seconds, 7 steps, 19.301719903945923 seconds per step \
Performance: rmse-mean = 0.5616448369465917

------------------------------------
##### Results for Add vectorization in Q_0
Runtime: 228.78 seconds, 17 steps, 13.46 seconds per step \
Performance: rmse-mean = 0.6174

------------------------------------
##### Results for Add vectorization in E-Step normalizing constant
Runtime: 28.37 seconds, 9 steps, 3.15 seconds per step \
Performance: rmse-mean = 0.5808

------------------------------------
##### Results for Add Suggested Initial Parameters (Zhang)
Runtime: 61.22 seconds, 19 steps, 3.22 seconds per step \
Performance: rmse-mean = 0.5625

------------------------------------
##### Results for Fixed Q_item
Runtime: 81.11 seconds, 12 steps, 6.76 seconds per step \
Optimal marginal Likelihood: -375.43, Estimated: -393.8, Initial -391.83
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.762005 |     0.512703 |     0.600404 |
| Girth     | 1.61804  |     0.750525 |   nan        |
| Initial   | 0.770281 |     0.227528 |     0.141421 |

------------------------------------
##### Results for Inproved GA stoppimg rule
Runtime: 42.54 seconds, 7 steps, 6.08 seconds per step \
Optimal marginal Likelihood: -378.68, Estimated: -391.87, Initial -378.08
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.845078 |     0.568092 |    0.167374  |
| Girth     | 0.930362 |     0.554795 |    0.141421  |
| direct    | 0.401578 |     0.609516 |    0.0526782 |
| Initial   | 0.509902 |     0.364352 |    0.141421  |

------------------------------------
##### Results for pycma ga used
Runtime: 489.8 seconds, 16 steps, 30.61 seconds per step \
Optimal marginal Likelihood: -381.34, Estimated: -414.28, Initial -379.77
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.942127 |     0.505808 |     0.215139 |
| Girth     | 1.42144  |     0.632431 |   nan        |
| Initial   | 0.509902 |     0.489947 |     0.141421 |

------------------------------------
##### Results for pycma, higher sigma0
Runtime: 282.36 seconds, 10 steps, 28.24 seconds per step \
Optimal marginal Likelihood: -368.93, Estimated: -399.56, Initial -369.53
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.898735 |     0.48221  |     0.195257 |
| Girth     | 1.79316  |     0.515695 |   nan        |
| direct    | 0.458306 |     0.262779 |     0.139413 |
| Initial   | 0.509902 |     0.273981 |     0.141421 |

------------------------------------
##### Results for Monte Carlo with seed
Runtime: 97.6 seconds, 9 steps, 10.84 seconds per step \
Optimal marginal Likelihood: -376.79, Estimated: -399.07, Initial -372.28
|           |      rmse_A |   rmse_delta |   rmse_sigma |
|:----------|------------:|-------------:|-------------:|
| Estimated |    1.09615  |     0.970204 |     0.120425 |
| Girth     |    1.45372  |     0.34729  |     0.141421 |
| direct    | 2935.78     |  1035.68     |     0.014461 |
| Initial   |    0.509902 |     0.445143 |     0.141421 |

------------------------------------
##### Results for Monte Cralo with fixed sample per em-step
Runtime: 1.22 seconds, 5 steps, 0.24 seconds per step \
Optimal marginal Likelihood: -381.06, Estimated: -373.26, Initial -377.69
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.500043 |     0.273166 |    0.0853068 |
| Girth     | 0.98542  |     0.334766 |    0.141421  |
| direct    | 0.57518  |     0.335156 |    0.0914744 |
| Initial   | 0.509902 |     0.431741 |    0.141421  |

------------------------------------
##### Results for Newton Raphson of Q_0 with approximate second derivative
Latent dimension: 2,  Item dimension: 6, sample size 100 \
Runtime: 2.2 seconds, 9 steps, 0.24 seconds per step \
Optimal marginal Likelihood: -389.58, Estimated: -375.96, Initial -386.41
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.452731 |     0.428534 |     0.011738 |
| early_initial | 0.509902 |     0.451316 |     0.212132 |
| girth         | 1.54173  |     0.489318 |     0.164638 |
| early_direct  | 1.10778  |     0.416042 |     0.189961 |

------------------------------------
##### Results for Quasi Monte-Carlo with increasing sample-size based on ttest'scipy
Latent dimension: 2,  Item dimension: 6, sample size 100 \
Runtime: 589.08 seconds, 52 steps, 11.33 seconds per step \
Optimal marginal Likelihood: -371.52, Estimated: -366.22, Initial -373.6
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.287802 |     0.302635 |    0.0019799 |
| early_initial | 0.509902 |     0.292938 |    0.212132  |
| girth         | 1.1188   |     0.385494 |    0.0478773 |
| early_direct  | 0.282345 |     0.319402 |    0.0080443 |

## Procedure for better runtime

1. Add Performance Benchmark (done)
2. Add Vectorization to q_item (done)
3. Add better Initialization (done)
3. Add Vectorization to q_0 (done)
4. Add q_0 Derivation 1. + BFGS (done)
5. Add q_0 Derivation 2. + Newton Raphson
6. Use cython (maybe)

## Experiment 1: MIRT-2PL Parameter Recovery

In [9]:
from unittest import result

def mirt_param_recovery(sample_size, item_dimension = 20, latent_dimension=3, q_type="seperated", girth=True, stop_threshold=0.2, ensure_id=False, q_share=0.0) -> dict:
    
    #Simulate Responses
    simulation = item_response_simulation(item_dimension=item_dimension, latent_dimension=latent_dimension)
    parameter_dict = simulation.set_up(q_structure=q_type, q_share=q_share, ensure_id=ensure_id)
    sample = simulation.sample(sample_size=sample_size)

    #Define Population
    #population = respondent_population(latent_dimension=latent_dimension)
    real_latent_cov = parameter_dict["real_early_parameters"]["person_parameters"]["covariance"]
    print("Real latent covariance: {0}".format(real_latent_cov))
    

    #Sample responses
    #response_simulation_obj = response_simulation(population=population, item_dimension=item_dimension)
    #response_simulation_obj.initialize_random_q_structured_matrix(structure=q_type, ensure_id=ensure_id)
    #early_item_parameters = response_simulation_obj.initialize_random_item_parameters()
    parameter_dict["real_early_parameters"].update({"item_dimension": item_dimension, "latent_dimension": latent_dimension})
    real_early_parameters = parameter_dict["real_early_parameters"]#{"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}}

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=latent_dimension, item_dimension=item_dimension, Q=real_early_parameters["item_parameters"]["q_matrix"])
    print("Covariance matrix is good: {0}".format(model.check_sigma(real_latent_cov)))
    model.initialize_from_responses(response_data=sample["early_responses"])
    initial_early_parameters = model.get_parameters()
    e_step = em_algorithm.e_step_ga_mml(model=model)
    m_step = em_algorithm.m_step_ga_mml(model)
    em = em_algorithm.em_algo(e_step=e_step, m_step=m_step, model=model)

    #Fit Model
    start_time = time.time()
    em.fit(sample["early_responses"], max_iter=100, stop_threshold=stop_threshold)
    run_time =  (time.time() - start_time)

    #Measure Performance
    early_estimated_item_parameters = em.model.item_parameters
    early_estimated_person_parameters = em.model.person_parameters

    #Create Baselines
    baselines = {"early_initial": {"parameters": initial_early_parameters}}
    if girth==True:
        baselines["girth"] = {}

    #Create results
    early_estimated_parameters = em.model.get_parameters()

    parameter_dict = create_parameter_dict(estimated_early_parameters=early_estimated_parameters,
                          real_early_parameters = real_early_parameters,
                          estimated_late_parameters=None, real_late_parameters=None)
    run_dict = {"early": {"runtime": run_time,
                "number_steps": em.n_steps}}
    performance_dict = create_performance_dict(parameter_dict=parameter_dict, run_dict=run_dict, sample=sample, baselines=baselines, model=model)

    return(performance_dict)

In [12]:
#Univariate IRT Recovery
result_dict = mirt_param_recovery(1000, q_type="singular", item_dimension=20, latent_dimension=1, stop_threshold=3)

Real latent covariance: [[1.]]
Covariance matrix is good: True
EM Iteration 2


c:\Users\Jesper\anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 150
Step: 2: current parameter_diff: 20.92439564532225, current marginal loglikelihood: -8910.492710324956
EM Iteration 3
Current Monte Carlo Sample size: 180
Step: 3: current parameter_diff: 9.809719715710592, current marginal loglikelihood: -8834.26793346509
EM Iteration 4
Current Monte Carlo Sample size: 216
Step: 4: current parameter_diff: 5.074222025392416, current marginal loglikelihood: -8804.164133349997
EM Iteration 5
Current Monte Carlo Sample size: 259
Step: 5: current parameter_diff: 2.297277979962996, current marginal loglikelihood: -8796.737923373841
EM Iteration 6
Current Monte Carlo Sample size: 310
Step: 6: current parameter_diff: 2.75208250396536, current marginal loglikelihood: -8795.999486702127
EM Iteration 7
Current Monte Carlo Sample size: 372
Step: 7: current parameter_diff: 0.3226698388793409, current marginal loglikelihood: -8793.867408990402
EM Iteration 8
Current Monte Carlo Sample size: 446
Step: 8: current parameter_diff: 1

c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:260: RuntimeWarning: divide by zero encountered in log
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:260: RuntimeWarning: invalid value encountered in multiply
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))


Step: 9: current parameter_diff: 2.434544830313956, current marginal loglikelihood: -8794.794419833572
EM Iteration 10
Current Monte Carlo Sample size: 642
Step: 10: current parameter_diff: 0.2447697163707434, current marginal loglikelihood: -8794.436511827398
EM Iteration 11
Current Monte Carlo Sample size: 770
Step: 11: current parameter_diff: 1.0991016801058746, current marginal loglikelihood: -8790.755641813757
EM Iteration 12
Current Monte Carlo Sample size: 924


c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\m_step_mirt_2pl.py:70: RuntimeWarning: overflow encountered in exp
  population = random.choices(population=population_base, weights=np.exp(


Step: 12: current parameter_diff: 3.0580715482334835, current marginal loglikelihood: -8819.665499989813
EM Iteration 13
Current Monte Carlo Sample size: 924
Step: 13: current parameter_diff: 3.6800389299500487, current marginal loglikelihood: -8912.144109280896
EM Iteration 14
Current Monte Carlo Sample size: 1108
Step: 14: current parameter_diff: 5.75286832444813, current marginal loglikelihood: -8830.867078491956
EM Iteration 15
Current Monte Carlo Sample size: 1108
Step: 15: current parameter_diff: 2.734590776760328, current marginal loglikelihood: -8874.298894890944
EM Iteration 16
Current Monte Carlo Sample size: 1108
Step: 16: current parameter_diff: 2.770682917042631, current marginal loglikelihood: -8891.349486464915
EM Iteration 17
Current Monte Carlo Sample size: 1329
Step: 17: current parameter_diff: 3.892323472787454, current marginal loglikelihood: -8844.21780037292
EM Iteration 18
Current Monte Carlo Sample size: 1594
Step: 18: current parameter_diff: 4.776805134864283, 

KeyboardInterrupt: 

In [11]:
print_result(result_dict, "Monte Carlo sample varying through ttest")

------------------------------------
##### Results for Monte Carlo sample varying through ttest
Latent dimension: 1,  Item dimension: 20, sample size 1000 \
Runtime: 21.38 seconds, 10 steps, 2.14 seconds per step \
Optimal marginal Likelihood: -10094.15, Estimated: -10067.58, Initial -10671.51
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  1.76372 |      3.08965 |            0 |
| early_initial |  3.07064 |      4.0076  |            0 |
| girth         |  1.4954  |      6.8503  |            0 |


### Univariate IRT/Full GA

------------------------------------
##### Item Dimension=12, sample_size=200: Results for Fixed sample per em-step
Runtime: 4.38 seconds, 7 steps, 0.63 seconds per step \
Optimal marginal Likelihood: -1284.1, Estimated: -1273.09, Initial -1330.43
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.508687 |     0.196944 |            0 |
| Girth     | 0.377882 |     2.65875  |            0 |
| direct    | 0.305489 |     0.318168 |            0 |
| Initial   | 1.5583   |     0.861273 |            0 |

------------------------------------
##### Item Dimension=20, sample_size=1000: Results for Fixed sample per em-step
Runtime: 85.11 seconds, 19 steps, 4.48 seconds per step \
Optimal marginal Likelihood: -9199.01, Estimated: -9183.03, Initial -9784.69
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.352003 |     0.578688 |            0 |
| Girth     | 0.468473 |     4.98511  |            0 |
| Initial   | 1.66615  |     2.01114  |            0 |

------------------------------------
##### Results for Monte Carlo sample varying through ttest
Latent dimension: 1,  Item dimension: 20, sample size 1000 \
Runtime: 31.86 seconds, 10 steps, 3.19 seconds per step \
Optimal marginal Likelihood: -9619.61, Estimated: -9601.49, Initial -9970.74
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.175897 |     0.223543 |            0 |
| early_initial | 1.21508  |     1.78456  |            0 |
| girth         | 0.327131 |     4.71968  |            0 |

In [9]:
performance_dict = mirt_param_recovery(sample_size=1000, item_dimension=20, latent_dimension=2, q_type="seperated", girth=True, stop_threshold=1)

Real latent covariance: [[1.         0.56672093]
 [0.56672093 1.        ]]
Covariance matrix is good: True
EM Iteration 2


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 300
Maximize Q-0
Maximize the Q_i's
Step: 2: current parameter_diff: 9.323444830286167, current marginal loglikelihood: -11834.985275023937
EM Iteration 3
Current Monte Carlo Sample size: 300
Maximize Q-0
Maximize the Q_i's
Step: 3: current parameter_diff: 5.460237617530897, current marginal loglikelihood: -11725.273459185906
EM Iteration 4
Current Monte Carlo Sample size: 300
Maximize Q-0
Maximize the Q_i's
Step: 4: current parameter_diff: 3.045977062824834, current marginal loglikelihood: -11695.808137179862
EM Iteration 5
Current Monte Carlo Sample size: 300
Maximize Q-0
Maximize the Q_i's
Step: 5: current parameter_diff: 1.0027257796336433, current marginal loglikelihood: -11687.160890125586
EM Iteration 6
Current Monte Carlo Sample size: 360
Maximize Q-0
Maximize the Q_i's
Step: 6: current parameter_diff: 0.4736484695346386, current marginal loglikelihood: -11687.932253978033
EM Iteration 7
Current Monte Carlo Sample size: 360
Maximize Q-0
Maximize

In [12]:
print_result(performance_dict, "Fixed random Covariance 1, girth included")

------------------------------------
##### Results for Fixed random Covariance 1, girth included
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 57.16 seconds, 14 steps, 4.08 seconds per step \
Optimal marginal Likelihood: -11668.66, Estimated: -11672.6, Initial -12307.41
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.183251 |    0.0861252 |    0.0215108 |
| early_initial | 0.786712 |    0.393667  |    0.0471788 |
| girth         | 1.28298  |    0.263374  |    0.350541  |


------------------------------------
##### Results for Seperated Q-Matrix
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 142.07 seconds, 26 steps, 5.46 seconds per step \
Optimal marginal Likelihood: -9384.38, Estimated: -10189.47, Initial -11459.9
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  4.00013 |      3.10678 |    0.0201383 |
| early_initial |  4.5183  |      2.5846  |    0.165491  |

------------------------------------
##### Results for BFGS for Q_0 with approximate Jacobi
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 61.55 seconds, 12 steps, 5.13 seconds per step \
Optimal marginal Likelihood: -9363.89, Estimated: -9711.58, Initial -10528.79
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  3.02507 |      1.83482 |    0.0645047 |
| early_initial |  2.06432 |      2.18337 |    0.289049  |

------------------------------------
##### Results for Monte Carlo sample varying through ttest
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 325.73 seconds, 42 steps, 7.76 seconds per step \
Optimal marginal Likelihood: -9929.77, Estimated: -10179.96, Initial -10767.41
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  1.25777 |     0.965894 |   0.00557421 |
| early_initial |  1.98812 |     1.68516  |   0.719705   |

------------------------------------
##### Results for Fixed random Covariance 1, girth included
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 57.16 seconds, 14 steps, 4.08 seconds per step \
Optimal marginal Likelihood: -11668.66, Estimated: -11672.6, Initial -12307.41
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.183251 |    0.0861252 |    0.0215108 |
| early_initial | 0.786712 |    0.393667  |    0.0471788 |
| girth         | 1.28298  |    0.263374  |    0.350541  |

In [10]:
performance_dict = mirt_param_recovery(sample_size=1000, item_dimension=20, latent_dimension=3, q_type="pyramid", girth=False, stop_threshold=3, ensure_id=True, q_share=0.6)

C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models\mirt_2pl.py:103: RuntimeWarning: overflow encountered in exp
  np.divide(1, np.add(1, np.exp(np.multiply(-1, linear_predictor)))))
c:\Users\Jesper\anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Real latent covariance: [[1.     0.7348 0.5198]
 [0.7348 1.     0.5761]
 [0.5198 0.5761 1.    ]]
Covariance matrix is good: True
EM Iteration 2
Current Monte Carlo Sample size: 600
Maximize Q-0
Maximize the Q_i's
Step: 2: current parameter_diff: 17.589659429116335, current marginal loglikelihood: -11681.696866135531
EM Iteration 3
Current Monte Carlo Sample size: 600
Maximize Q-0
Maximize the Q_i's
Step: 3: current parameter_diff: 6.6248661211591, current marginal loglikelihood: -11434.243215445382
EM Iteration 4
Current Monte Carlo Sample size: 600
Maximize Q-0
Maximize the Q_i's
Step: 4: current parameter_diff: 3.745292330440992, current marginal loglikelihood: -11346.077090722109
EM Iteration 5
Current Monte Carlo Sample size: 600
Maximize Q-0
Maximize the Q_i's
Step: 5: current parameter_diff: 2.866350978862611, current marginal loglikelihood: -11296.555396972382
EM Iteration 6
Current Monte Carlo Sample size: 600
Maximize Q-0
Maximize the Q_i's
Step: 6: current parameter_diff: 4.8

c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models\mirt_2pl.py:103: RuntimeWarning: overflow encountered in exp
  np.divide(1, np.add(1, np.exp(np.multiply(-1, linear_predictor)))))


In [11]:
print_result(performance_dict, "With Covariance Matrix partially aligned to Q-matrix")

------------------------------------
##### Results for With Covariance Matrix partially aligned to Q-matrix
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 304.59 seconds, 30 steps, 10.15 seconds per step \
Optimal marginal Likelihood: -11254.37, Estimated: -10937.57, Initial -12526.56
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  51.7548 |      3.42392 |     0.756073 |
| early_initial |  51.5531 |      3.86416 |     0.116728 |


------------------------------------
##### Results for Pyramid Q-Matrix
Latent dimension: 3,  Item dimension: 20, sample size 200 \
Runtime: 121.22 seconds, 46 steps, 2.64 seconds per step \
Optimal marginal Likelihood: -2144.15, Estimated: -2155.08, Initial -2699.63
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  11.0034 |      1.77461 |     0.889828 |
| early_initial |  10.5116 |      2.15986 |     0.522199 |

------------------------------------
##### Results for Pyramid Q-Matrix
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 313.22 seconds, 51 steps, 6.14 seconds per step \
Optimal marginal Likelihood: -9452.24, Estimated: -9813.87, Initial -11216.48
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  5.34115 |      1.4068  |     0.425982 |
| early_initial |  5.98238 |      2.25109 |     0.234288 |

------------------------------------
##### Results for Pyramid Q-Matrix
Latent dimension: 3,  Item dimension: 30, sample size 2000 \
Runtime: 1521.51 seconds, 76 steps, 20.02 seconds per step \
Optimal marginal Likelihood: -30844.13, Estimated: -32521.34, Initial -37390.6
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  2.55959 |       1.9952 |    0.314331  |
| early_initial |  3.32034 |       3.514  |    0.0239562 |

------------------------------------
##### Results for BFGS for Q_0 with approximated gradient
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 350.28 seconds, 64 steps, 5.47 seconds per step \
Optimal marginal Likelihood: -10498.87, Estimated: -11303.0, Initial -12740.14
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  10.2704 |     0.68597  |    0.318713  |
| early_initial |  10.4626 |     0.846172 |    0.0895367 |

------------------------------------
##### Results for BFGS for Q_0 with exact gradient
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 152.91 seconds, 31 steps, 4.93 seconds per step \
Optimal marginal Likelihood: -11106.69, Estimated: -11211.51, Initial -12370.23
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  3.5074  |      2.54509 |     0.21123  |
| early_initial |  3.51819 |      2.66313 |     0.319647 |

------------------------------------
##### Results for Newton-Raphson for Q_0 with approximated second derivative
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 352.93 seconds, 62 steps, 5.69 seconds per step \
Optimal marginal Likelihood: -10237.22, Estimated: -10482.36, Initial -12569.49
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  5.23694 |      2.45127 |     0.508499 |
| early_initial |  5.36172 |      3.29141 |     0.640209 |

In [13]:
performance_dict = mirt_param_recovery(sample_size=1000, item_dimension=30, latent_dimension=3, q_type="full", girth=False, stop_threshold=5, q_share=0.0)

Real latent covariance: [[1.     0.6388 0.    ]
 [0.6388 1.     0.    ]
 [0.     0.     1.    ]]
Covariance matrix is good: True
EM Iteration 2


c:\Users\Jesper\anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 600
Step: 2: current parameter_diff: 20.550243550546323, current marginal loglikelihood: -18149.686153495782
EM Iteration 3
Current Monte Carlo Sample size: 600
Step: 3: current parameter_diff: 10.215343616726535, current marginal loglikelihood: -17883.31370973797
EM Iteration 4
Current Monte Carlo Sample size: 600
Step: 4: current parameter_diff: 7.723302485746889, current marginal loglikelihood: -17771.151983294196
EM Iteration 5
Current Monte Carlo Sample size: 600
Step: 5: current parameter_diff: 3.8262687990163315, current marginal loglikelihood: -17686.13343764576
EM Iteration 6
Current Monte Carlo Sample size: 600
Step: 6: current parameter_diff: 3.8036503763306806, current marginal loglikelihood: -17652.39359227588
EM Iteration 7
Current Monte Carlo Sample size: 600
Step: 7: current parameter_diff: 3.0244823809405057, current marginal loglikelihood: -17629.497205774373
EM Iteration 8
Current Monte Carlo Sample size: 600
Step: 8: current paramete

Exception: New Covariance not positive semidefinite

In [12]:
print_result(performance_dict, "With BFGS as covariance optimization Algorithm")

------------------------------------
##### Results for With BFGS as covariance optimization Algorithm
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 160.7 seconds, 20 steps, 8.04 seconds per step \
Optimal marginal Likelihood: -11451.44, Estimated: -11746.37, Initial -12757.23
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  4.12413 |      1.19297 |     0.539132 |
| early_initial |  4.06187 |      1.22103 |     0.159225 |


### Procedure for better model performance:

1. Implement MIRT Baseline (done)
2. Test GA with quadratic function (done)
3. use pycma (done)
4. Test mirt-functions (response-matrix-probability, done)
5. Read on MC MIRT (MC-Errorbounds usage or quasi-MC or importance sampling)
6. Use importance sampling
7. Implement Newton Raphson

### Estimeted Parameters

In [11]:
from scipy.optimize import approx_fprime

def func(C_vector):
    dim = np.sqrt(C_vector.shape).astype(np.int)[0]
    C = C_vector.reshape((dim, dim))
    sigma = np.matmul(C, C.transpose())
    return(sigma.flatten())

approx_fprime(f=func, xk=np.ones(2*2))

#Ich brauche wohl irgendeine Art von Matrixprodukt

array([[2., 2., 0., 0.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [0., 0., 2., 2.]])

In [12]:
approx_fprime(f=func, xk=np.arange(0,4
))

array([[1.49011612e-08, 2.00000001e+00, 0.00000000e+00, 0.00000000e+00],
       [2.00000000e+00, 3.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [2.00000000e+00, 3.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.00000000e+00, 6.00000000e+00]])